#LỌC CỘNG TÁC DỰA TRÊN ITEM

Là thuật toán gợi ý dựa trên sự tương đồng của item mục tiêu $t$ với các item khác trong tập dữ liệu. Cụ thể, thuật toán sẽ đo độ tương đồng của item $t$ với các item khác, sau đó lọc ra $k$ items tương đồng nhất với item $t$ và sử dụng rating của user $u$ cho $k$ item này để dự đoán số điểm user $u$ đánh giá cho item $t$.

### **Ví dụ**
(Bài tập 2 trang 69 sách Recommender System của Springer)

Ta có một ma trận rating gồm 5 users (theo hàng) và 6 (theo cột) items như sau:

ItemId->|i0|i1|i2|i3|i4|i5|
-|:-:|:-:|:-:|:-:|:-:|:-:|
u0|5|6|7|4|3|Nan|
u1|4|Nan|3|Nan|5|4|
u2|Nan|3|4|1|1|Nan|
u3|7|4|3|6|Nan|4|
u4|1|Nan|3|2|2|5|

*Yêu cầu*: dự đoán giá trị bị khuyết của user 1 bằng thuật toán lọc cộng tác dựa trên item.  Sử dụng hàm tương đồng cosin hiệu chỉnh (adjusted cosine similarity)
Chọn $k=2$.






#### **Các bước thực hiện**

1. Chuẩn hóa dữ liệu:
  - Tính giá trị trung bình của mỗi user dựa trên rating quan sát được
$$\mu_u = \frac{\sum_{k \in I_u} r_{uk}}{|I_u|}, \forall u \in \{1,...,m\}.$$
trong đó $I_u$ là tập hợp gồm các chỉ số item mà được đánh giá bởi user $u$.
  - Trừ số điểm rating của từng item mà user $u$ đã đánh giá cho giá trị trung bình của user $u$.
  $$s_{uj} = r_{uj} - \mu_u, \forall u \in \{1,...,m\}.$$
2. Tính độ tương đồng cosine (cosine similarity) và độ tương đồng cosine có hiệu chỉnh (adjusted cosine similarity) giữa các items:
$$\text{Cosine}(i,j) = \frac{ \sum_{u \in U_i \cap U_j} r_{ui} \cdot r_{uj} }{\sqrt{ \sum_{u \in U_i \cap U_j} r_{ui}^2 } \cdot \sqrt{\sum_{u \in U_i \cap U_j} r_{uj}^2 } }$$

$$\text{AdjustedCosine}(i,j) = \frac{ \sum_{u \in U_i \cap U_j} s_{ui} \cdot s_{uj} }{\sqrt{ \sum_{u \in U_i \cap U_j} s_{ui}^2 } \cdot \sqrt{\sum_{u \in U_i \cap U_j} s_{uj}^2 } }$$

3. Chọn top k items tương đồng với item mục tiêu $t$ mà đã được user $u$ đánh giá, tập này được kí hiệu là $Q_t(u)$.
4. Dự đoán rating của user $u$ đối với item mục tiêu $t$:
$$\hat r_{ut} = \frac{\sum_{j \in Q_t(u)} \text{AdjustedCosine}(j,t) \cdot r_{uj} }{\sum_{j \in Q_t(u)} |\text{AdjustedCosine}(j,t)|}.$$


Cụ thể, trong bài toán này, cần tính độ tương tự cosine có hiệu chỉnh của item $t$ với các item khác trong ma trận rating và chọn ra $k=2$ item tương đồng nhất, sau đó dự đoán số điểm rating mà user 1 có khả năng rate.

ItemId->|i0|i1|i2|i3|i4|i5|Mean Rating|
-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
u0|5|6|7|4|3|Nan|5|
u1|4|Nan|3|Nan|5|4|4|
u2|Nan|3|4|1|1|Nan|2.25|
u3|7|4|3|6|Nan|4|4.8|
u4|1|Nan|3|2|2|5|2.6|

Ma trận sau khi chuẩn hóa

ItemId->|i0|i1|i2|i3|i4|i5|
-|:-:|:-:|:-:|:-:|:-:|:-:|
u0|0|1|2|-1|-2|Nan|
u1|0|Nan|-1|Nan|1|0|
u2|Nan|0.75|1.75|-1.25|-1.25|Nan|
u3|2.2|-0.8|-1.8|1.2|Nan|-0.8|
u4|-1.6|Nan|0.4|-0.6|-0.6|2.4|
AdjCos(i1,j)|-0.625|1|0.998|-0.976|-0.996|1.0|
AdjCos(i3,j)|0.791|-0.976|-0.975|1.0|0.943|-0.707|

*Cách tính độ tương đồng cosine có hiệu chỉnh:*

$\text{AdjustedCosine}(i1,i0) = \frac{ \sum_{u \in U_1 \cap U_0} s_{u1} \cdot s_{u0} }{\sqrt{ \sum_{u \in U_1 \cap U_0} s_{u1}^2 } \cdot \sqrt{\sum_{u \in U_1 \cap U_0} s_{u0}^2 } } = \frac{(1*0)+(-0.8)*2.2}{\sqrt{1^2+(-0.8)^2}\cdot \sqrt{0^2+2.2^2}} = -0.625$

$\text{AdjustedCosine}(i1,i2) = \frac{1*2+(0.75)*(1.75)+(-0.8)*(-1.8)}{\sqrt{1^2+0.75^2+(-0.8)^2}\cdot \sqrt{2^2+1.75^2+(-1.8)^2}} = 0.998$

*Chọn k item tương đồng nhất với item t:*
- t = 1: (item 5 và item 2)
- t = 3: (item 4 và item 0)

*Dự đoán rating của user 1 đối với item 1 và item 3*

$$\hat r_{1,1} = \frac{\text{AdjustedCosine}(1,5) \cdot r_{15} + \text{AdjustedCosine}(1,2) \cdot r_{12}} { \text{AdjustedCosine}(1,5) + \text{AdjustedCosine}(1,2)} = \frac{1.0*4 + 0.998*3}{1+0.998} = 3.5$$

$$\hat r_{1,3} = \frac{0.943*5+0.791*4}{0.943+0.791} = 4.5$$

####**Sử dụng Python**

Có thể chia nhỏ bài toán thành các bước sau:

- Bước 1: Chuẩn hóa dữ liệu (mean-centering)
- Bước 2: Tính ma trận tương đồng giữa các item, sử dụng adjusted cosine similarity
- Bước 3: Lọc những item mà user mục tiêu đã đánh giá trong quá khứ.
- Bước 4: Lọc những item mà user mục tiêu chưa đánh giá
- Bước 5: Chọn k items tương đồng với từng item đã được lọc ra ở bước 4
- Bước 6: Dự đoán số điểm mà user mục tiêu có thể đánh giá cho từng item ở bước 4
- Bước 7: Sắp xếp số điểm theo thứ tự từ cao đến thấp

In [40]:
# import libraries
import pandas as pd
import numpy as np

In [74]:
# Ma trận rating
mx = np.array([[5,6,7,4,3,np.nan],
              [4,np.nan,3,np.nan,5,4],
              [np.nan,3,4,1,1,np.nan],
              [7,4,3,6,np.nan,4],
              [1,np.nan,3,2,2,5]])

# Chuyển ma trận A thành dataframe
example_df = pd.DataFrame(mx, index=['u0','u1','u2','u3','u4'], columns=['i0','i1','i2','i3','i4','i5'])

In [75]:
def cosine(item_i, item_j):
  # Tìm index mà item_i và item_j đều được rate bởi tất cả các user
  mutual_index = [u for u in item_i.index if not np.isnan(item_i[u]) and not np.isnan(item_j[u])]

  dividend = 0
  i_divisor = 0
  j_divisor = 0

  # Tính độ tương đồng cosine
  for index in mutual_index:
    dividend += item_i[index] * item_j[index]
    i_divisor += pow(item_i[index], 2)
    j_divisor += pow(item_j[index], 2)

  divisor = np.sqrt(i_divisor) * np.sqrt(j_divisor) # Mẫu số của cosine

  if divisor != 0:
    return round(dividend / divisor, 3) # trả về độ tương đồng nếu mẫu số != 0

  return 0 # trả về 0 nếu mẫu số = 0


def cosine_mx(df):
  m, n = df.shape
  corr_mx = np.identity(n) # ma trận đơn vị có số chiều bằng số lượng item
  for i in range(n-1):
    for j in range(i+1,n):
      corr_mx[i,j] = corr_mx[j,i] = cosine(df.iloc[:, i], df.iloc[:, j])
  corr_mx = pd.DataFrame(corr_mx, index=df.columns, columns=df.columns)

  return corr_mx


In [78]:
# Item-based recommendation function
def item_based_rec(data, target_user, k=2):
  """Input: data là tập dữ liệu có số dòng là userID và số cột là itemID"""

  # print("Dữ liệu gốc: ")
  # display(data)

  # Bước 1: Mean-centering
  normalized_df = data.subtract(data.mean(axis=1), axis='rows')

  # Bước 2: Ma trận tương đồng giữa các items
  corr_mx = cosine_mx(normalized_df)
  # print("\n")
  # print("Ma trận tương đồng: ")
  # display(corr_mx)

  # Bước 3: Những item mà user mục tiêu đã đánh giá
  target_user_rated = data[data.index==target_user].dropna(axis=1) # items đã đánh giá
  target_user_rated_df = target_user_rated.T.reset_index().rename(columns={target_user:'rating', 'index':'item'}) # chuyển vị, reset lại index và đổi tên cột

  # print("\n")
  # print(f"Các item mà user mục tiêu đã đánh giá:")
  # display(target_user_rated_df)

  # Bước 4: Những item mà user mục tiêu đã đánh giá
  target_user_unrated = data.drop(target_user_rated.columns, axis=1, errors='ignore') # drop những item đã đánh giá

  # print("\n")
  # print(f"Các item mà user mục tiêu chưa đánh giá: {target_user_unrated.columns.values}")

  # Khởi tạo dictionary to lưu những item chưa được đánh giá và số điểm dự đoán tương ứng
  rating_prediction ={}

  # Lặp qua từng item trong tập item chưa được đánh giá
  for picked_item in target_user_unrated:
    # Bước 5: Tính điểm tương đồng của picked_item với các item khác
    picked_item_sim_score = corr_mx[[picked_item]].reset_index().rename(columns={'index':'item', picked_item:'similarity_score'})
    # print("\n")
    # print(f"Độ tương đồng của {picked_item} với các item khác: ")
    # display(picked_item_sim_score)

      # Sắp xếp k items có điểm tương đồng theo thứ tự từ cao đến thấp
    target_user_rated_sim = pd.merge(left=target_user_rated_df,
                                     right=picked_item_sim_score,
                                     on='item',
                                     how='inner')\
                                     .sort_values('similarity_score', ascending=False)[:k]
    # print("\n")
    # print(f"Các item tương đồng nhất với item {picked_item}: ")
    # display(target_user_rated_sim)
    # print("\n")

    # Bước 6: Dự đoán số điểm mà user mục tiêu có thể đánh giá cho picked_item
    #predicted_rating = round((target_user_rated_sim['rating']*target_user_rated_sim['similarity_score']).sum()/target_user_rated_sim['similarity_score'].sum(),3)
    predicted_rating = round(np.average(target_user_rated_sim['rating'],
                                        weights=target_user_rated_sim['similarity_score']), 3)
    # Lưu predicted rating vào dictionary
    rating_prediction[picked_item] = predicted_rating

  # Bước 7: Chuyển dictionary thành dataframe và sắp xếp dữ liệu theo prediction_score
  pred_score = (pd.DataFrame(rating_prediction.items(), columns=['item', 'pred_score'])
                      .sort_values(by='pred_score', ascending=False))

    # Trả về dataframe gồm item mà target_user chưa đánh giá cùng với số điểm rating dự đoán
  return pred_score

In [79]:
# Get recommendations
target_user='u1'
recommended_item = item_based_rec(example_df, target_user, k=2)
recommended_item

Dữ liệu gốc: 


,i0,i1,i2,i3,i4,i5
u0,5.0,6.0,7.0,4.0,3.0,NaN
u1,4.0,NaN,3.0,NaN,5.0,4.0
u2,NaN,3.0,4.0,1.0,1.0,NaN
u3,7.0,4.0,3.0,6.0,NaN,4.0
u4,1.0,NaN,3.0,2.0,2.0,5.0




Ma trận tương đồng: 


,i0,i1,i2,i3,i4,i5
i0,1.000,-0.625,-0.583,0.791,0.259,-0.814
i1,-0.625,1.000,0.998,-0.976,-0.996,1.000
i2,-0.583,0.998,1.000,-0.975,-0.984,0.452
i3,0.791,-0.976,-0.975,1.000,0.943,-0.707
i4,0.259,-0.996,-0.984,0.943,1.000,-0.514
i5,-0.814,1.000,0.452,-0.707,-0.514,1.000




Các item mà user mục tiêu đã đánh giá:


...





Độ tương đồng của i3 với các item khác: 


,item,similarity_score
0,i0,0.791
1,i1,-0.976
2,i2,-0.975
3,i3,1.000
4,i4,0.943
5,i5,-0.707




Các item tương đồng nhất với item i3: 


,item,rating,similarity_score
2,i4,5.0,0.943
0,i0,4.0,0.791


,item,pred_score
1,i3,4.544
0,i1,3.501


### **Bài tập**

Tập dữ liệu có kích thước 20x20, gồm 20 users (dòng) và 20 movies (20 cột).

Link file: https://docs.google.com/spreadsheets/d/10ZmaLuHhrhSklR1aIpPUcpVd1j7lF-DU/edit?usp=sharing&ouid=115422524039268283690&rtpof=true&sd=true

Trong bài tập này, tính toán sự tương đồng giữa các movie bằng cách:
- Sử dụng hàm tương đồng cosine không hiệu chỉnh
- Sử dụng hàm tương đồng cosine có hiệu chỉnh

Yêu cầu:
1. Đề xuất top 5 bộ phim tương đồng nhất với phim "Toy Story", theo mức độ tương đồng cosine giảm dần.

2. Đề xuất top 5 phim cho user 5277 bằng cách dự đoán số điểm rating mà user 5277 có thể đánh giá cho các phim mà user này chưa xem.

3. Nhận xét kết quả khi dùng 2 hàm cosine trên.


In [57]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

# Thay đổi đương dẫn
import os
os.chdir("drive/My Drive/KHTN/RecSys/data")

# Print out the current directory
!pwd

Mounted at /content/drive
/content/drive/My Drive/KHTN/RecSys/data


In [58]:
# Load the dataset
data = pd.read_excel('data_02.xlsx')
data.rename(columns={"User": "userId"}, inplace=True)
data.set_index('userId', inplace = True)
data.head()


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,1: Toy Story (1995),1210: Star Wars: Episode VI - Return of the Jedi (1983),356: Forrest Gump (1994),"318: Shawshank Redemption, The (1994)","593: Silence of the Lambs, The (1991)",3578: Gladiator (2000),260: Star Wars: Episode IV - A New Hope (1977),2028: Saving Private Ryan (1998),296: Pulp Fiction (1994),1259: Stand by Me (1986),2396: Shakespeare in Love (1998),2916: Total Recall (1990),780: Independence Day (ID4) (1996),541: Blade Runner (1982),1265: Groundhog Day (1993),"2571: Matrix, The (1999)",527: Schindler's List (1993),"2762: Sixth Sense, The (1999)",1198: Raiders of the Lost Ark (1981),34: Babe (1995)
userId,,,,,,,,,,,,,,,,,,,,
755,2.0,5.0,2.0,NaN,4.0,4.0,1.0,2.0,NaN,3.0,2.0,NaN,5.0,2.0,5.0,4.0,2.0,5.0,NaN,NaN
5277,1.0,NaN,NaN,2.0,4.0,2.0,5.0,NaN,NaN,4.0,3.0,2.0,2.0,NaN,2.0,NaN,5.0,1.0,3.0,NaN
1577,NaN,NaN,NaN,5.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,3.0
4388,2.0,3.0,NaN,NaN,NaN,1.0,NaN,3.0,4.0,NaN,NaN,4.0,NaN,3.0,5.0,NaN,5.0,1.0,1.0,2.0
1202,NaN,3.0,4.0,1.0,4.0,1.0,4.0,4.0,NaN,1.0,5.0,1.0,NaN,4.0,NaN,3.0,5.0,5.0,NaN,NaN
